# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 5. Работа с API ВКонтакте: собираем посты со стены, комментарии к постам и информацию пользователях

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [2]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.
По [инструкции](https://allatambov.github.io/pypolit/vk-auth.pdf) мы можем получить доступ к API, создадим приложение и скопируем его ID, чтобы получить ссылку для токена доступа:

In [3]:
app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 51660535


Переходим по ссылке выше и копируем ссылку (полностью!) из адресной строки:

In [7]:
full_link = input()

Разбиваем полученную ссылку (сначала по `access_token=`, потом часть после `=` по `&`) и забираем токен в чистом виде:

In [8]:
token = full_link.split("access_token=")[1].split("&")[0]

Ура! Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1: выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [скалодрома Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [9]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [10]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [11]:
req_wall = requests.get(main_wall, params = params_wall)

In [12]:
json_wall = req_wall.json()
#json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [13]:
nposts = json_wall['response']['count']
print(nposts)

1742


Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [14]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [16]:
i = items_wall[2]
i

{'inner_type': 'wall_wallpost',
 'donut': {'is_donut': False},
 'comments': {'can_post': 1, 'count': 0, 'groups_can_post': True},
 'marked_as_ads': 0,
 'short_text_rate': 0.8,
 'hash': '5feRmK_fVWyXwNGvSQ-nu7Wkej0',
 'type': 'post',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1708097659,
    'id': 457244846,
    'owner_id': -38936316,
    'access_key': '88a7d053faaa0aea7b',
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun9-41.userapi.com/impg/tFMy6c11P0g3S2Uf3TVdepGUxM9hhP9T8TMTLw/2fWzaOvI0ZU.jpg?size=75x75&quality=95&sign=76d6e483798341727d0861dcd11cfeac&c_uniq_tag=mHUTkgoddewQIJaTVLAroBVs096Q0ds_UZ89O8r7ebg&type=album'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun9-41.userapi.com/impg/tFMy6c11P0g3S2Uf3TVdepGUxM9hhP9T8TMTLw/2fWzaOvI0ZU.jpg?size=130x130&quality=95&sign=84297a7c16e5f3788c2131bbf66875da&c_uniq_tag=is9YyXL1gZnG9snUB_Vkx6TPznMiIzAQs_GcrfKAM1w&type=album'},


### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [17]:
id_ = i["id"]
date = i["date"]
text = i["text"]
nlikes = i["likes"]["count"]
nreposts = i["reposts"]["count"]
ncomments = i["comments"]["count"]

print(id_, date, nlikes, nreposts, ncomments)
print(text)

4410 1708097659 14 0 0
💥 С 19 по 27 февраля - фестиваль Boulderhouse Fest в [club38936316|@rzclimbing]!
Около 50 трасс! 💪

1️⃣ Первая накрутка: понедельник 19.02, начало лазания с 09:00
2️⃣ Вторая накрутка - вторник 20.02, начало лазания с 15:00 

Лазать фестивальные трассы можно до 27.02 20:30 включительно. 

Награждение проводится 27 февраля в 21:00! 🍾

💪 А в 20:00 для вас будет подготовлена ПОЛОСА ПРЕПЯТСТВИЙ, можете больше не завидовать детям, а попробовать сами: длинные перехваты, динамические движения и просто веселье!

🔥 Лазить можно в любые дни, самостоятельно отмечая пролазы. 

🔥 Зачет в категориях новички (CLIMB) и любители (CLIMB+).

Стартового взноса нет, просто списывайте занятия с абонемента или оплатите обычный вход на скалодром 💚

📌 Заявиться на участие в фестивале можно у администратора на скалодроме «Rock Zona Boulderhouse» в любой из дней проведения фестиваля и получить карточку участника и уникальный код.


### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [18]:
iterate = nposts // 100 + 1

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [19]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [20]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    # обновляем список items_all
    # обновляем парметр offset - увеличиваем на 100
    items_all.extend(items_wall_long)
    params_wall_long["offset"] = params_wall_long["offset"] + 100
    
    time.sleep(1.5)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


Проверяем длину списка – все ли посты собраны:

In [21]:
len(items_all)

1742

Не отбираем ничего на этом этапе, просто превращаем список словарей в датафрейм:

In [22]:
df = pd.DataFrame(items_all)
df.head(10)

,inner_type,donut,comments,marked_as_ads,short_text_rate,hash,type,carousel_offset,attachments,date,...,post_source,post_type,reposts,text,views,edited,copy_history,geo,signer_id,zoom_text
0,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,mf3h7SfvWi5LnfRQ2UfD3-2JscQ,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1708357405,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 0, 'user_reposted': 0}","🔥🔥🔥 Друзья, наш фестиваль Boulderhouse.Fest ст...",{'count': 155},NaN,NaN,NaN,NaN,NaN
1,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,Nd94usWVdsEmTa24a6hmxB2s3Ko,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1708354461,...,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",,{'count': 195},NaN,NaN,NaN,NaN,NaN
2,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,5feRmK_fVWyXwNGvSQ-nu7Wkej0,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1708097659,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 0, 'user_reposted': 0}",💥 С 19 по 27 февраля - фестиваль Boulderhouse ...,{'count': 534},1.708127e+09,NaN,NaN,NaN,NaN
3,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 1, 'groups_can_post':...",0,0.8,Qe0Jjk1YXCbZxZcv_2SdXSnKp84,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1708002654,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 2, 'user_reposted': 0}","Новички, охваченные энтузиазмом, часто попадаю...",{'count': 1075},NaN,NaN,NaN,NaN,NaN
4,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,EYLMGqUFAaJ3DQcnWu1vDQ6rk6w,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1707990427,...,{'type': 'vk'},post,"{'count': 1, 'user_reposted': 0}",,{'count': 425},NaN,NaN,NaN,NaN,NaN
5,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 3, 'groups_can_post':...",0,0.8,bb8ez30eWrwStF-RNLWd8mavfEA,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1707849256,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 0, 'user_reposted': 0}",Друзья! \nБудьте внимательны - с 15 по 18 февр...,{'count': 1385},NaN,NaN,NaN,NaN,NaN
6,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 4, 'groups_can_post':...",0,0.8,az20aD-Z1heagC_txkfDxjnlTao,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1707751452,...,{'type': 'vk'},post,"{'count': 2, 'user_reposted': 0}",,{'count': 859},NaN,NaN,NaN,NaN,NaN
7,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,1hNngvRVtwmqnLxqZAWx54_GlDQ,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1707413546,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 0, 'user_reposted': 0}",🔥 Наши подготовщики каждую неделю добавляют но...,{'count': 1095},NaN,NaN,NaN,NaN,NaN
8,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,AAGDCqDCOnvZMBsQkjZtMYw-l-s,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1707413480,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 3, 'user_reposted': 0}",Скалолазам невысокого роста часто бывает нелег...,{'count': 1254},NaN,NaN,NaN,NaN,NaN
9,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,Z-zCRERQnm6nGLbxYYCfFAsdUjQ,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1707397215,...,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",,{'count': 562},NaN,NaN,NaN,NaN,NaN


### Задача 4

Создайте на основе датафрейма `df` новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы.

In [25]:
small = df[["id", "date", "text", "likes", "reposts", "views", "comments"]]

In [26]:
# пишем lambda-функцию, которая принимает на вход ячейку (x)
# и забирает оттуда значение по ключу count
# для nviews пишем условие с if-else на случай пустых ячеек –
# для старых постов данных о просмотрах нет

small["nlikes"] = small["likes"].apply(lambda x: x["count"])
small["nreposts"] = small["reposts"].apply(lambda x: x["count"])
small["ncomments"] = small["comments"].apply(lambda x: x["count"])
small["nviews"] = small["views"].apply(lambda x: x["count"] if x is dict else x)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [27]:
small.drop(columns = ["reposts", "comments", "likes"], inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


**Дополнение.** Преобразуем дату из формата POSIX во что-то более привычное:

In [28]:
# unit = 's': переводим из метки, которая является числом секунд с 1 января 1970

small["date_time"] = pd.to_datetime(small['date'], unit = 's')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
# забираем части даты-времени в виде строки (тип string)
# %Y – год в четырехзначном виде, %y – год в двузначном виде
# %m – месяц в числовом виде
# %d – день в числовом виде
# %A – день недели полностью, %a – день недели сокращенно
# %H, %M, %S – часы, минуты, секунды

small["year"] = small["date_time"].dt.strftime('%Y')
small["month"] = small["date_time"].dt.strftime('%m')
small["day"] = small["date_time"].dt.strftime('%d')
small["week_day"] = small["date_time"].dt.strftime('%A')
small["time"] = small["date_time"].dt.strftime('%H:%M:%S')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

### Задача 5

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [31]:
with_comm = small[small["ncomments"] > 0] 

## Часть 2: собираем комментарии к постам

В целях экономии времени не будем брать весь датафрейм `with_comm`, для примера отберем первые 10 строк:

In [32]:
with_comm10 = with_comm.head(10)

Заберем id постов и преобразуем их в список:

In [33]:
ids = list(with_comm10["id"])

Возьмем один id для примера и сформулируем запрос для получения комментариев к посту с этим id:

In [34]:
i = 3782

# метод wall.getComments
# owner_id: id сообщества
# post_id: id поста
# v: версия API, все та же
# access_token: токен доступа, все тот же
# count: число комментариев, берем 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10, max за раз

main_comm = "https://api.vk.com/method/wall.getComments"
params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}

Формулируем запрос, отправляем и смотрим на результат в виде словаря:

In [35]:
req = requests.get(main_comm, params = params_comm)
req.json()

{'response': {'count': 4,
  'items': [{'id': 3783,
    'from_id': 527923327,
    'date': 1669311085,
    'text': 'Мотивация не нужна, нужна дисциплина.',
    'post_id': 3782,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 3784,
       'from_id': 2311030,
       'date': 1669314785,
       'text': '[id527923327|Николай], 👍',
       'post_id': 3782,
       'owner_id': -38936316,
       'parents_stack': [3783],
       'reply_to_user': 527923327,
       'reply_to_comment': 3783}],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}},
   {'id': 3785,
    'from_id': 2847724,
    'date': 1669373425,
    'text': 'Мотивация тоже нужна)',
    'post_id': 3782,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 3788,
       'from_id': 527923327,
       'date': 1669400413,
       'text': '[id2847724|Татьяна], мотивация лишь временный фактор, дисциплина - вечна. Дис

### Задача 6

Давайте считать, что 100 самых новых комментариев и 10 ответов на них нам достаточно (если нет – аргумент `offset` есть, задача аналогична предыдущей по сбору постов). Повторите эту операцию выше для всех id в списке `ids` и сформируйте список с результатами. Преобразуйте результат в датафрейм `df_comm`. 

In [36]:
# по аналогии с циклом ранее, только ничего не увеличиваем
# подставляем на место post_id значение i

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [37]:
df_comm = pd.DataFrame(comments_all)

### Задача 7

Объедините датафрейм с собранными комментариями `df_comm` и датафрейм `with_comm10` через функцию `merge()` из `pandas`.

In [38]:
# переименовываем столбцы в with_comm10, чтобы 
# столбец с id поста назывался везде одинаково
# inplace = True: сохраняем изменения

with_comm10.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm10.head(2)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,post_id,date,text,views,nlikes,nreposts,ncomments,nviews,date_time,year,month,day,week_day,time
3,4406,1708002654,"Новички, охваченные энтузиазмом, часто попадаю...",{'count': 1075},13,2,1,{'count': 1075},2024-02-15 13:10:54,2024,02,15,Thursday,13:10:54
5,4402,1707849256,Друзья! \nБудьте внимательны - с 15 по 18 февр...,{'count': 1385},11,0,3,{'count': 1385},2024-02-13 18:34:16,2024,02,13,Tuesday,18:34:16


In [39]:
# объединяем with_comm10 и df_comm по столбцу post_id
# left: к левому датафрейму (with_comm10) подтягиваем данные из правого (df_comm)

final = with_comm10.merge(df_comm, on = "post_id", how = "left")

## Часть 3:  собираем информацию о пользователях

Для примера выберем пользователя с id 20473269 (пока не из датафрейма, это всего лишь я):

In [40]:
j = 20473269

Сформируем запрос:

In [41]:
# метод users.get
# все знакомые параметры
# в fields перечисляем дополнительню информацию,
# которую хотим получить
# перечисляем через запятую без пробелов

main_user = "https://api.vk.com/method/users.get"
params_user = {"access_token" : token,
               "v" : v,
               "user_id" : j,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

{'response': [{'id': 20473269,
   'bdate': '25.3.1994',
   'city': {'id': 1, 'title': 'Москва'},
   'home_town': 'Москва',
   'universities': [{'city': 1,
     'country': 1,
     'education_status': 'Студентка (бакалавр)',
     'education_status_id': 3,
     'graduation': 2016,
     'id': 128,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)'}],
   'first_name': 'Алла',
   'last_name': 'Тамбовцева',
   'can_access_closed': True,
   'is_closed': False}]}

### Задача 8

Выберите какого-нибудь пользователя, оставившего комментарий, из датафрейма и запросите по нему интересующую информацию.

In [43]:
user = final["from_id"][0]

In [44]:
params_user = {"access_token" : token,
               "v" : v,
               "user_id" : user,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

{'response': [{'id': 260091507,
   'home_town': 'Москва',
   'universities': [{'city': 1,
     'country': 1,
     'id': 332,
     'name': 'РГТЭУ (бывш. ЗИСТ, МКИ, МКУ, МГУК)'}],
   'first_name': 'Владимир',
   'last_name': 'Цепаев',
   'can_access_closed': True,
   'is_closed': False}]}